In [1]:
import github3
from dotenv import load_dotenv, dotenv_values
import os

load_dotenv()

github = github3.login(token=os.getenv('GITHUB_TOKEN'))

In [2]:
js_issues = github.issues_on(username='open-telemetry', repository='opentelemetry-js', state='open')

In [3]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import instructor
from itertools import islice

class Issue(BaseModel):
    title: str = Field(..., description="The title of the issue.")
    url: str = Field(..., description="The URL of the issue.")
    libraries: List[str] = Field(..., description="A list of libraries mentioned in the issue.")
    positive_reactions: int = Field(..., description="The number of positive reactions on the issue.")
    negative_reactions: int = Field(..., description="The number of negative reactions on the issue.")

client = instructor.from_openai(OpenAI())

responses = []

for issue in islice(js_issues, 10):
    json_content = issue.as_json()
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are a system that parses GitHub issues and extracts data from them."
            },
            {
                "role": "user",
                "content": json_content
            }
        ],
        response_model=Issue
    )
    responses.append(resp)

In [4]:
import pprint

pprint.pprint(responses)

[Issue(title='Instrumenting node build chains', url='https://github.com/open-telemetry/opentelemetry-js/issues/4872', libraries=['Webpack', 'Rollup'], positive_reactions=0, negative_reactions=0),
 Issue(title='Unable to set object as span attribute value', url='https://github.com/open-telemetry/opentelemetry-js/issues/4871', libraries=['OpenTelemetry.js'], positive_reactions=0, negative_reactions=0),
 Issue(title='Update Typescript', url='https://github.com/open-telemetry/opentelemetry-js/issues/4870', libraries=['typescript', 'downlevel-dts', 'typesVersions'], positive_reactions=0, negative_reactions=0),
 Issue(title='Logs Stabilization Plan', url='https://github.com/open-telemetry/opentelemetry-js/issues/4869', libraries=['API', 'SDK'], positive_reactions=0, negative_reactions=0),
 Issue(title="couchbase instrumentation doesn't not support?", url='https://github.com/open-telemetry/opentelemetry-js/issues/4868', libraries=['couchbase', 'opentelemetry'], positive_reactions=0, negative_